In [180]:
import numpy as np;
import pandas as pd;

In [181]:
# 各类函数
class function:
    def __init__(self):
        pass;
    def activate(self,f,x):
        if f == 'relu':
            return self.relu(x);
        if f == 'softmax':
            return self.softmax(x);
        if f == 'sigmoid':
            return self.sigmoid(x);
        if f == 'x':
            return x;
    def loss(self,f,y,label):
        if f == 'MSE':
            return 1/y.shape[0] * 1/2* np.sum((y - label)**2);
        if f == 'Cross_Entropy':
            res = - np.sum(label*np.log(y),axis=1);
            return 1/y.shape[0] * np.sum(res);
        if f == 'Real_Cross_Entropy':
            res = - np.sum(label*np.log(y)+(1-label)*np.log(1-y),axis=1);
            return 1/y.shape[0] * np.sum(res);
    def gra_func(self,f,x,y=None):
        if f == 'relu':
            return self.gra_relu(x);
        if f =='softmax':
            return self.gra_softmax(x);
        if f == 'sigmoid':
            return self.gra_sigmoid(x);
        if f == 'x':
            return 1;
    def gra_loss(self,f,y,label):
        if f == 'MSE':
            return (y-label);
        if f == 'Cross_Entropy':
            return -label/y;
        if f == 'Real_Cross_Entropy':
            return -(label/y-(1-label)/(1-y));
    def relu(self,x):
        res = np.abs(x);
        res = (res + x)/2;
        return res;
    def softmax(self,x):
        x = x.T - np.max(x,axis=1).T;
        x = x.T # 减去最大，防止溢出 
        res = np.exp(x).T/np.sum(np.exp(x),axis=1).T
        return res.T;
    def sigmoid(self,x):
        r1 = 1/(1+np.exp(-self.relu(x)));
        r2 = np.exp(-self.relu(-x))/(1+np.exp(-self.relu(-x)));
        return (r1+r2-1/2);
    def gra_relu(self,x):
        res = np.abs(x);
        res = np.sign(res + x);
        return res;
    def gra_softmax(self,x):
        x = x.T - np.max(x,axis=1).T;
        x = x.T
        out = np.exp(x).T/np.sum(np.exp(x),axis=1).T
        out = out.T
        return (out - out**2);
    def gra_sigmoid(self,x):
        r1 = np.exp(-self.relu(x))/(1+np.exp(-self.relu(x)))**2;
        r2 = np.exp(-self.relu(-x))/(1+np.exp(-self.relu(-x)))**2;
        return (r1+r2-1/4);

In [182]:
# 定义层
class class_layers:    
    def __init__(self):
        pass;
    def flatten(self,input_shape = None):
        # 该方法将任意输入形状数据展平;
        return {'type':'flatten','input_shape':input_shape};
    def dense(self,num_nets = None,activation = 'relu',learning_rate=0.01): 
        # 一层全连接神经网络
        return {'type':'dense','num_nets':num_nets,'activation_func':activation,'learning_rate':learning_rate};

In [183]:
class data_process:
    def __init__(self):
        pass;
    # 随机打乱输入数据，将数据划分为训练/测试集
    def random_division(self,x,y=None,test_rate = 0):
        x = np.array(x);
        # 数据总长度
        n = np.shape(x)[0];
        # 生成下表序列并随即打乱
        sequence = np.array(range(n));
        np.random.shuffle(sequence);
        # 测试集长度
        n_test = int(np.ceil(n * test_rate));
        # 测试集序列与训练集序列
        se_test = sequence[:n_test];
        se_train = sequence[n_test:];
        # 训练集
        train_data = x[se_train];
        # 测试集
        test_data = x[se_test];
        # 有监督学习的情况
        if not type(y) == type(None):
            y = np.array(y);
            train_label = y[se_train];
            test_label = y[se_test];
            if not test_rate == 0:
                return (train_data,train_label,test_data,test_label);
            else:
                return (train_data,train_label);
        # 无监督情况
        if not test_rate == 0:
            return (train_data,test_data);
        else:
            return (train_data);
    # 归一化
    def normalization(self,x_train,x_test=None,norm_type= 'max_abs'):
        # 最大绝对值放缩
        if norm_type == 'max_abs':
            max_abs = np.max(np.abs(x_train),axis=0)
            x_train = x_train/max_abs;
            if not type(x_test) == type(None):
                x_test = x_test / max_abs;
        # 最大最小值归一化
        if norm_type == 'max_min':
            max_v = np.max(x_train);
            min_v = np.min(x_train);
            x_train = (max_v - x_train) / (max_v - min_v);
            if not type(x_test) == type(None):
                x_test = (max_v - x_test) / (max_v - min_v);
        # 标准化
        if norm_type == 'norm':
            mean_v = np.mean(x_train,axis=0);
            std_v = np.std(x_train,axis=0);
            x_train = (max_v - mean_v) / std_v;
            if not type(x_test) == type(None):
                x_test = (max_v - mean_v) / std_v;
        # 函数返回
        if not type(x_test) == type(None):
            return (x_train,x_test);
        return x_train;
    # 将数字标签转化为one hot标签
    def onehot_label(self,label):
        max_num = np.max(label);
        res = [];
        for k in range(len(label)):
            resk = np.zeros((max_num+1,));
            resk[label[k]] = 1;
            res.append(resk);
        return np.array(res);
    # 在定义batch时，将原数据分段
    def batch_slice(self,batch,n):
        train_list = [];
        if batch <= 0 or batch >n:
            batch = n;
            train_list.append([0,n]);
        else:
            m = n//batch;
            for k in range(m):
                train_list.append([k*batch,(k+1)*batch]);
            if not n%batch == 0:
                train_list.append([(k+1)*batch,n]);
        return train_list;
    # 返回多分类的onehot表示
    def arg_max(y):
        tar = np.argmax(y,axis=1);
        res = np.zeros_like(y);
        for i in range(y.shape[0]):
            res[i,tar[i]] = 1;
        return res;

In [184]:
class fit_info:
    def __init__(self):
        pass;      
    def state_display(self,state,timer,loss=-1,epoch=1,epoches=1,batch=-1,all=-1,metrics=None,metrics_value=None):
        # title
        if state == 1:
            print('Epoch {}/{}'.format(epoch+1,epoches));
            self.n_str = 0;
        # loading bar
        if state == 2:
            self.loading_bar(timer,timer/all,all,loss,metrics=metrics,metrics_value=metrics_value);
    # 进度条
    def loading_bar(self,timer,process,all,loss,metrics=None,metrics_value=None):
        n = int(process*30) + 1;
        m = 30-n;
        s = '{}/{} '.format(timer,all);
        s += '[' + '='*n + '-'*m + ']';
        s += '\tloss: {:.2f}'.format(loss);
        if not type(metrics) == type(None):
            s += ' {}: {:.2f}'.format(metrics,metrics_value);
        n = len(s);
        s += ' '*(self.n_str - n);
        print('\r',end='')
        print(s,end='');
        if n > self.n_str:
            self.n_str = n;
    # 分类网络的评价指标
    def evaluate(self,y,label,metrics='Accuracy'):
        confusion_matrix = np.dot(label.T,y);
        cm = confusion_matrix;
        # 对角线元素+1 防止溢出
        cm = cm + np.eye(cm.shape[0]);
        N = y.shape[0] + cm.shape[0];
        n = y.shape[1];
        T = np.diagonal(cm);
        A = np.trace(cm)/N;
        Pk = T/np.sum(cm,axis=0);
        P = np.sum(Pk)/n;
        Rk = T/np.sum(cm,axis=1).T;
        R = np.sum(Rk)/n;
        F1 = 2*P*R/(P+R);
        if metrics == 'Accuracy':
            return A;
        if metrics == 'Precision':
            return P;
        if metrics == 'Recall':
            return R;
        if metrics == 'F1':
            return F1;

In [185]:
class model:
    # 类初始化
    def __init__(self):
        self.model_state = {'compile_model':False,'loading_data':False};
        self.structure = {};
        self.info = {};
        self.num_layers = 0;
        self.parameter = {};
        self.layers = class_layers();
        self.res_layer ={};
        self.median_value = {};
        self.error = None;
        self.nets = [];
        self.in_shape = [];
        self.out_shape = [];
        self.func = function();
        self.data_proc = data_process();
        self.fit_state = fit_info();

    # 向模型加载数据
    def load_data(self,
    train_data=None,
    train_label=None,
    validation_data=None,
    validation_label=None,
    random_train_set=True,
    auto_test_set=False,
    test_proportion=0.1,
    data_normalization=False,
    normalization_type='max_abs'
    ):
        # 自动划分测试集合
        if auto_test_set == True:
            if not type(train_label) == type(None):
                # 有监督情况
                (train_data,train_label,validation_data,validation_label) = self.data_proc.random_division(train_data,train_label,test_rate=test_proportion);
            else:
                # 无监督情况
                (train_data,validation_data) = self.data_proc.random_division(train_data,test_rate=test_proportion);
        # 训练数据随机化
        elif random_train_set == True:
            if not type(train_label) == type(None):
                # 有监督情况
                (train_data,train_label) = self.data_proc.random_division(train_data,train_label,test_rate=0);
            else:
                # 无监督情况
                (train_data) = self.data_proc.random_division(train_data,test_rate=0);
        # 数据归一化
        if data_normalization == True:
            if not type(validation_data) == type(None):
                (train_data,validation_data) = self.data_proc.normalization(train_data,validation_data,normalization_type);
            else:
                train_data = self.data_proc.normalization(train_data,None,normalization_type);
        # 记录加载训练集信息
        if not type(train_data) == type(None):
            self.train_data = train_data;
            self.model_state.update({'loading_train_data':True});
            self.info.update({'num_train_data':train_data.shape[0]});
            self.n_train = train_data.shape[0];
        if not type(train_label) == type(None):
            self.train_label = train_label;
            self.model_state.update({'loading_train_label':True});
            self.d_train = train_data.shape[1:];
        # 记录加载验证集信息
        if not type(validation_data) == type(None):
            self.validation_data = validation_data;
            self.model_state.update({'loading_validation_data':True});
            self.info.update({'num_validation_data':validation_data.shape[0]});
            self.n_val = validation_data[0];
        if not type(validation_label) == type(None):
            self.validation_label = validation_label;
            self.model_state.update({'loading_validation_label':True});
            self.d_val = validation_data[1:];
        # 记录数据加载信息
        if self.model_state['loading_train_data'] == True and self.model_state['loading_train_label'] == True:
            self.model_state['loading_data'] = True;

    # 描述新的网络结构
    def define(self,BP_structure = None):
        if not type(BP_structure) == type(None):
            self.layer_type = [];
            for layer in BP_structure:
                self.structure.update({self.num_layers:layer});
                self.layer_type.append(layer['type']);
                self.num_layers += 1;
    # 向已有结构新添层
    def add(self,new_layer):
        self.structure.update({self.num_layers:new_layer});
        self.num_layers += 1;
    # 网络编译初始化参数
    def compile_init(self):
        self.x = [];
        self.y = [];
        self.w = [];
        self.b =[];
        self.nets = [];
        self.rate = [];
        self.activation_function =[];
        self.delta = [];
    # 编译网络输入输出的维度信息
    def compile_dim_info(self):
        n = self.n_train;
        d = self.d_train;
        out_shape = d;
        for k in range(self.num_layers):
            layer = self.structure[k];
            if layer['type'] == 'flatten':
                in_shape = out_shape;
                out_shape = 1;
                for p in d:
                    out_shape *= p;
                self.nets.append(np.array([]));
                self.in_shape.append(in_shape);
                self.out_shape.append(out_shape);
                continue;
            if layer['type'] == 'dense':
                in_shape = out_shape;
                out_shape = layer['num_nets'];
                self.nets.append(layer['num_nets']);
                self.in_shape.append(in_shape);
                self.out_shape.append(out_shape);
                continue;
    # 编译网络所需要的中间量
    def compile_param_info(self):
        for k in range(self.num_layers):
            layer = self.structure[k];
            if layer['type'] == 'flatten':
                self.x.append(np.array([]));
                self.y.append(np.array([]));
                self.w.append(np.array([]));
                self.b.append(np.array([]));
                self.rate.append(np.array([]));
                self.activation_function.append(np.array([]));
                self.delta.append(np.array([]));
                continue;
            if layer['type'] == 'dense':
                N1 = self.in_shape[k];
                N2 = self.out_shape[k];
                w = np.random.randn(N1,N2);
                b = np.random.randn(1,N2);
                self.x.append(np.array([]));
                self.y.append(np.array([]));
                self.w.append(w);
                self.b.append(b);
                self.rate.append(layer['learning_rate']);
                self.activation_function.append(layer['activation_func']);
                self.delta.append(np.array([]));
                continue;

        if not N2 == len(self.train_label[0]):
            raise Exception("UNFITTED OUTPUT LAYER");
    # 编译该神经网络
    def compile(self,loss_function = 'MSE',optimizer='SGD',metrics = None):
        self.loss_function = loss_function;
        self.optimizer = optimizer;
        self.metrics = metrics;
        self.compile_init();
        # 维度信息
        self.compile_dim_info();
        # 参数信息
        self.compile_param_info();
        self.model_state['compile_model']=True;
    # 模型训练
    def fit(self,epoches=1,batch = -1):
        # 防止未加载数据或未编译网络时训练
        if self.model_state['compile_model'] == False or self.model_state['loading_data'] == False:
            raise Exception("UNINITIALIZED MODEL: (loading data:{}, compile model:{})".format(
                self.model_state['loading_data'],self.model_state['compile_model']));
        # BGD优化训练
        if self.optimizer == 'BGD':
            self.BGD(epoches,batch);
    # 前向传播
    def forward_propagation(self,x0):
        # 输入层
        if self.layer_type[0] == 'flatten':
            x = x0;
            y = [];
            for i in range(x.shape[0]):
                xk = x[i];
                y.append(xk.reshape(-1));
        elif self.layer_type[0] == 'dense':
            x = np.dot(x0,self.w[0]) + self.b[0];
            y = self.func.activate(self.activation_function[0],x);
        self.x[0] = x;
        self.y[0] = np.array(y);
        # 隐藏层
        for k in range(1,len(self.nets)):
            if self.layer_type[k] == 'flatten':
                x = self.y[k-1];
                y = [];
                for i in range(len(x)):
                    xk = x[i];
                    y.append(xk.reshape(-1));
                self.x[k] = x;
                self.y[k] = np.array(y);
            elif self.layer_type[k] == 'dense':
                x = np.dot(self.y[k-1],self.w[k]) + self.b[k];
                y = self.func.activate(self.activation_function[k],x);
                self.x[k] = x;
                self.y[k] = y;
    # 反向传播
    def backward_propagation(self,label):
        # 输出层情况
        k = len(self.nets) - 1;
        delta = self.func.gra_loss(self.loss_function,self.y[k],label) * self.func.gra_func(self.activation_function[k],self.x[k]);
        db = np.sum(delta,axis=0).reshape(1,-1);
        dw = np.dot(self.y[k-1].T,delta);
        self.b[k] -= self.rate[k]*db;
        self.w[k] -= self.rate[k]*dw;
        self.delta[k] = delta;
        k -= 1;
        # 隐藏层
        while(k>0):
            if self.layer_type[k] == 'flatten':
                delta = self.delta[k+1];
                delta = delta.reshape(-1,self.in_shape[k]);
                self.delta[k] = delta;
            elif self.layer_type[k] == 'dense':
                delta = np.dot(self.delta[k+1],self.w[k+1].T) * self.func.gra_func(self.activation_function[k],self.x[k]);
                db = np.sum(delta,axis = 0).reshape(1,-1);
                dw = np.dot(self.y[k-1].T,delta);
                self.b[k] -= self.rate[k]*db;
                self.w[k] -= self.rate[k]*dw;
                self.delta[k] = delta;
            k -= 1;

    # BGD优化器
    def BGD(self,epoches,batch = -1):
        if batch <= 0 or batch > self.n_train:
            batch = self.n_train;
        trian_list = self.data_proc.batch_slice(batch,self.n_train)
        for p in range(epoches):
            self.timer = 0;
            self.fit_state.state_display(state=1,timer=self.timer,epoch=p,epoches=epoches);
            for k in trian_list:
                self.timer += 1;
                x0 = self.train_data[k[0]:k[1]];
                label = self.train_label[k[0]:k[1]];
                # 正向传播
                self.forward_propagation(x0);
                # 计算误差
                loss = self.func.loss(self.loss_function,self.y[-1],label);
                # 反向传播
                self.backward_propagation(label);
                # 输出误差
                metrics_value = None;
                if not type(self.metrics) == type(None):
                    metrics_value = self.fit_state.evaluate(self.y[-1],label,self.metrics)
                self.fit_state.state_display(state=2,timer=self.timer,loss=loss,all=len(trian_list),metrics = self.metrics,metrics_value = metrics_value);
            print('');
    # 训练状态信息输出


In [186]:
# 加载数据
names_to_label = {'Iris-setosa': 0, 'Iris-versicolor': 1, 'Iris-virginica': 2};
label_to_names = {value: key for key, value in names_to_label.items()}
df = pd.read_csv('iris.data', header=None)
xs = df.iloc[:, :4].values
ts = np.array([names_to_label[name] for name in df.iloc[:, -1]])
ts = ts.reshape(-1,1);

In [187]:
# 调取模组
pp = model();
# onehot label
ys = pp.data_proc.onehot_label(ts);
# 加载数据
pp.load_data(
    train_data = xs,
    train_label = ys,
    auto_test_set=True,# 从训练集划分出测试集
    test_proportion=0.1,# 划分的测试集的比例
    data_normalization=True,# 对数据归一化
    normalization_type='max_abs'# 归一化的类型
);

In [188]:
# 定义网络
rate = 1e-4;
pp.define([
    pp.layers.flatten(),
    pp.layers.dense(num_nets=32,activation='relu',learning_rate=rate),
    pp.layers.dense(num_nets=128,activation='relu',learning_rate=rate),
    pp.layers.dense(num_nets=64,activation='sigmoid',learning_rate=rate),
    pp.layers.dense(num_nets=3,activation='softmax',learning_rate=rate)
])
# 编译网络
pp.compile(loss_function = 'MSE',optimizer='BGD',metrics='Accuracy');

In [198]:
# 训练
pp.fit(epoches=100,batch=-1);
# epoches 总的迭代次数
# batch 每次训练的样本数量
# batch <= 1 or batch >= 所有样本：训练所有样本

Epoch 1/100
1/1 [===============================]	loss: 0.03 Accuracy: 0.87
Epoch 2/100
1/1 [===============================]	loss: 0.03 Accuracy: 0.87
Epoch 3/100
1/1 [===============================]	loss: 0.03 Accuracy: 0.87
Epoch 4/100
1/1 [===============================]	loss: 0.03 Accuracy: 0.87
Epoch 5/100
1/1 [===============================]	loss: 0.03 Accuracy: 0.87
Epoch 6/100
1/1 [===============================]	loss: 0.03 Accuracy: 0.87
Epoch 7/100
1/1 [===============================]	loss: 0.03 Accuracy: 0.87
Epoch 8/100
1/1 [===============================]	loss: 0.03 Accuracy: 0.87
Epoch 9/100
1/1 [===============================]	loss: 0.03 Accuracy: 0.87
Epoch 10/100
1/1 [===============================]	loss: 0.03 Accuracy: 0.87
Epoch 11/100
1/1 [===============================]	loss: 0.03 Accuracy: 0.87
Epoch 12/100
1/1 [===============================]	loss: 0.03 Accuracy: 0.87
Epoch 13/100
1/1 [===============================]	loss: 0.03 Accuracy: 0.87
Epoch 14

In [ ]:
z